In [11]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import cv2
# Define a custom dataset class for GRAZPEDWRI-DX
class GrazpedwriDataset(Dataset):
    def __init__(self, image_filenames, labels, transform=None):
        self.image_filenames = image_filenames
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image = cv2.imread(self.image_filenames[idx], cv2.IMREAD_GRAYSCALE)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Load image filenames and labels (assuming you have already loaded them)
# Replace with actual image filenames and labels
dataset = pd.read_csv("grazpedwri_dx_dataset.csv")
label = pd.read_csv('labels.csv')
image_filenames = dataset['filestem']+'.png'
labels = label['label']  # Binary labels (e.g., 0 for non-fracture, 1 for fracture)

# Split data into train and validation sets
train_filenames, val_filenames, train_labels, val_labels = train_test_split(
    image_filenames, labels, test_size=0.2, random_state=42
)

# Define data transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

# Create custom datasets
train_dataset = GrazpedwriDataset(train_filenames, train_labels, transform=transform)
val_dataset = GrazpedwriDataset(val_filenames, val_labels, transform=transform)

# Create data loaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained ResNet-152
model = models.resnet152(pretrained=True)
num_features = model.fc.in_features

# Modify the classifier for binary classification
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze()
            val_loss += criterion(outputs, labels.float()).item()

    print(f"Epoch [{epoch+1}/{num_epochs}] - Val Loss: {val_loss/len(val_loader):.4f}")

print("Training complete!")

# Save the trained model
torch.save(model.state_dict(), "resnet152_grazpedwri.pth")
print("Model saved as resnet152_grazpedwri.pth")
